In [1]:
import numpy as np
import pandas as pd
import PIL.Image as img
import os

In [2]:
covidli = "COVID"
covid_olmayan = "non-COVID"

In [3]:
def dosya(yol):
    dosyalar = []
    for f in os.listdir(yol):
        if f.endswith('.jpg') or f.endswith('.png'):
            dosyalar.append(os.path.join(yol, f))
    return dosyalar


In [4]:
def veri_donustur(klasör_adi,sinif_adi):
    goruntuler = dosya(klasör_adi)
    goruntu_sinif = []

    for goruntu in goruntuler:
        goruntu_oku = img.open(goruntu).convert("L")
        goruntu_boyutu = goruntu_oku.resize((28,28))
        goruntu_donuşturme = np.array(goruntu_boyutu).flatten()
        
        if sinif_adi == "covidli":
            veriler= np.append(goruntu_donuşturme,[0])
        elif sinif_adi == "covidli olmayan":
            veriler = np.append(goruntu_donuşturme,[1])
        else:
            continue
        goruntu_sinif.append(veriler)
    return goruntu_sinif

In [5]:
covidli_veri = veri_donustur(covidli, "covidli")
covidli_veri_df = pd.DataFrame(covidli_veri)

In [6]:
#covidli_veri

In [7]:
print(covid_olmayan)
print(covidli)

non-COVID
COVID


In [8]:
covidli_veri_df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,79,170,209,208,210,210,210,210,212,216,...,231,233,227,223,218,210,207,211,211,0
1,68,63,110,209,206,226,242,231,226,229,...,231,236,222,211,203,205,205,216,181,0
2,62,104,207,205,213,235,233,229,227,227,...,240,226,214,205,204,205,209,214,152,0
3,65,66,65,169,209,208,231,239,232,225,...,235,221,210,204,205,209,218,149,79,0
4,66,66,66,170,210,208,229,241,232,225,...,242,230,217,207,205,206,211,210,139,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,193,194,194,192,195,217,239,237,230,213,...,221,209,218,248,230,208,196,195,195,0
1248,193,195,193,194,218,246,229,172,145,149,...,223,224,228,230,216,201,196,196,197,0
1249,194,201,185,195,200,231,243,225,198,206,...,223,221,219,212,201,197,197,206,171,0
1250,208,168,167,196,204,230,238,228,222,234,...,219,213,200,197,205,193,123,69,68,0


In [9]:
covidli_olmayan_veri = veri_donustur(covid_olmayan, "covidli olmayan")
covidli_olmayan_veri_df = pd.DataFrame(covidli_olmayan_veri)

In [10]:
covidli_olmayan_veri_df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,66,62,112,195,209,206,207,206,205,205,...,231,211,212,239,238,207,213,196,109,1
1,184,206,208,219,228,232,233,233,233,234,...,168,143,179,227,227,218,222,235,251,1
2,62,78,189,206,210,227,234,233,234,237,...,110,89,127,195,236,236,235,238,234,1
3,65,65,162,208,205,219,232,234,233,234,...,220,221,226,222,225,233,242,236,231,1
4,59,132,210,206,222,234,234,233,237,241,...,208,199,223,223,224,231,238,237,232,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,67,117,204,232,245,236,219,230,227,201,...,248,236,230,225,218,209,210,188,104,1
1225,63,64,76,160,209,213,216,232,242,244,...,231,230,242,254,235,212,208,193,96,1
1226,63,68,95,194,211,219,230,243,246,225,...,235,232,233,238,223,208,211,126,77,1
1227,66,70,72,112,205,216,233,245,240,225,...,231,248,251,232,224,213,210,173,80,1


In [11]:
tum_veri= pd.concat([covidli_veri_df,covidli_olmayan_veri_df])

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [13]:
x = tum_veri.iloc[:,:-1]
y= tum_veri.iloc[:,-1]

In [14]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size=0.2, random_state=27)

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
sc = StandardScaler()

In [17]:
xtrain1 = sc.fit_transform(xtrain)
xtest1 = sc.transform(xtest)

In [18]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=22)
rf.fit(xtrain1,ytrain)

RandomForestClassifier(max_depth=5, random_state=22)

In [19]:
rf.score(xtest1,ytest)

0.869215291750503

In [20]:
from sklearn.metrics import confusion_matrix
yhead = rf.predict(xtest1)
cm = confusion_matrix(ytest,yhead)
cm

array([[225,  51],
       [ 14, 207]])

In [21]:
def covid_tahmini_yap(model, resim_yolu, scaler=None):
    # Resmi oku ve özellik vektörüne dönüştür
    resim = img.open(resim_yolu).convert("L")
    resim = resim.resize((28, 28))
    resim_duzen = np.array(resim).flatten()
    
    if scaler:
        resim_duzen = scaler.transform([resim_duzen])  # Veri normalizasyonu
    
    # Tahmin yap
    tahmin = model.predict(resim_duzen)
    
    # Sonucu ekrana yazdır
    if tahmin == 0:
        print("Resimde COVID-19 var.")
    else:
        print("Resimde COVID-19 yok.")


In [22]:
covid_tahmini_yap(rf, 'deneme/noncovid.png', sc)

Resimde COVID-19 yok.


In [23]:
import pickle

# Eğitilmiş modeli diske kaydetme
with open('new_model1.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [24]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(sc, f)